# Galileo Orbiter Solid State Imaging (SSI) Experiment

The Galileo Orbiter Solid State Imaging (SSI) Experiment was a camera system on the Galileo spacecraft that captured high-resolution images of Jupiter and its moons. The SSI instrument provided valuable data for studying the geology, atmospheres, and surfaces of these celestial bodies.

In [ ]:
# imports
from loguru import logger
from planetarypy.pds.index_main import Index
from planetarypy.pds import get_index, list_available_indexes


In [ ]:
logger.enable("planetarypy")

## The many index files

The `get_edr_index` function of the `go_ssi` module retrieves the orinigal plain EDR index file from the PDS archive. This file contains some metadata about all available EDR files, including their filenames on the PDS server, observation times, and target names, but unfortunately yet observation latitudes and longitudes.

For this there is a set of several other index files, which can be found [here](https://pds-rings.seti.org/viewmaster/metadata/GO_0xxx/GO_0999).

You can retrieve these files using the `get_index` function from the `planetarypy.pds` module. For example, to get the index file for Ganymede (GO_0999):

In [ ]:
list_available_indexes()

In [ ]:
ind = get_general_index()

In [ ]:
# frameno = "C0382996800R"
frameno = "62100R"  # multiple
image_id = "OPN0138"  # multiple

In [ ]:
edr = EDRSet.from_frameno(frameno)
# edr = EDRSet.from_image_id("OPN0138")

In [ ]:
edr.meta.query("TARGET_NAME == 'GANYMEDE'")

In [ ]:
edr.preview_full_url

In [ ]:
edr.download(refresh=True)

In [ ]:
BROWSE_URLBASE / edr.meta.loc["subpath"].values

In [ ]:
edr.target

In [ ]:
edr = EDR("0265R")


In [ ]:
edr.subpath

In [ ]:
edr.download()

In [ ]:
edr.exists

In [ ]:
edr.download_url.with_suffix(".LBL")

In [ ]:
edr.image_path

In [ ]:
from pathlib import Path

In [ ]:
(edr.save_folder / Path("EDR") / edr.urlpath).values

In [ ]:
[Path(i).parent for i in "EDR" / edr.meta.loc["subpath"]]

In [ ]:
edr.urlpath

In [ ]:
DOWNLOAD_URLBASE / edr.urlpath

In [ ]:
pd.set_option("display.max_columns", None)

In [ ]:
"EDR" + "/" + edr.volume + "/" + edr.image_id

In [ ]:
edr = EDR.from_filespec("C0382996800R")

In [ ]:
edr.urlpath

or, to get the `moon_summary` file:

In [ ]:
moon = get_index("go.ssi.moon_summary")

In [ ]:
body = get_index("go.ssi.body_summary")

In [ ]:
supp = get_index("go.ssi.supplemental_index")

In [ ]:
ind[ind.FILE_SPECIFICATION_NAME.str.contains("3126")]

In [ ]:
ind[ind.FILE_SPECIFICATION_NAME.str.contains("C0382996800R.IMG")]

In [ ]:
supp.iloc[0].T

Interestingly, most data had a target name of `EARTH` ?

## Frames, IMAGE_IDs a.k.a. PICNO, and OBSERVATION_IDs

What is important to understand is the subdivision of the data.
The smallest unit is an EDR file, which contains a single image or frame, which in some papers is referred to via a frame number.

Then, an `IMAGE_ID` can have one or more frames associated with it. The `IMAGE_ID` is also referred to as `PICNO` in some papers.

Finally, an `OBSERVATION_ID` can have one or more `IMAGE_ID`s associated with it.

Let's see how we can show this with the EDR index file.

In [ ]:
ind.index.name = "Index row"

In [ ]:
imgids = ind.groupby("IMAGE_ID").size().sort_values(ascending=False)

In [ ]:
imgids[imgids > 1]

In [ ]:
import pandas as pd

paper_images = pd.read_csv(
    "./Galileo_SSI_image_list_kersten_mosaic_ganymede.txt", sep=" "
)

In [ ]:
paper_images.head()

In [ ]:
def filter_imgid(imgid):
    mask1 = ind["FILE_SPECIFICATION_NAME"].str.contains(imgid, case=False)
    mask2 = ind["FILE_SPECIFICATION_NAME"].str.contains("GANYMEDE")
    return ind[mask1 & mask2]


def count_occurences(imgid):
    return filter_imgid(imgid).shape[0]

In [ ]:
paper_images["occurences"] = paper_images.ImageID.map(count_occurences)

In [ ]:
paper_images.query("occurences > 1")

In [ ]:
filtered = []
for img in paper_images.ImageID:
    df = filter_imgid(img)
    if df.shape[0] == 0:
        logger.warning(f"No match for {img}")
    else:
        filtered.append(df)

In [ ]:
pd.options{"display.max_columns":None}

In [ ]:
filter_imgid("1100R").T

In [ ]:
pd.concat(filtered).to_csv("ganymede_ssr_images_that_match_paper_id.csv", index=False)

In [ ]:
paper_images.shape

In [ ]:
edr = EDR("10C0008")

In [ ]:
edr.volume

In [ ]:
edr.urlpath

In [ ]:
edr.browse_url

In [ ]:
edr.download_url

In [ ]:
edr.label_url

In [ ]:
edr.local_folder

In [ ]:
edr.image_path

In [ ]:
edr.label_path

In [ ]:
edr.download(refresh=True)

In [ ]:
edr.preview_small_url

In [ ]:
ganymedes = ind.query("TARGET_NAME == 'GANYMEDE'")

In [ ]:
ganymedes.VOLUME_ID.value_counts()

In [ ]:
logger.disable("planetarypy")

In [ ]:
import pandas as pd

failed_urls = []
for frameno in tqdm(ganymedes.FILE_SPECIFICATION_NAME):
    edr = EDR(frameno, save_dir="/remote/trove/geo/planet/Jupiter/SSI")
    edr.download(leave_tqdm=False)
    if hasattr(edr, "failed_urls"):
        d = dict(image_id=img_id, failed_urls=edr.failed_urls)
        failed_urls.append(d)

In [ ]:
import json

In [ ]:
failed_urls[0]

In [ ]:
json.dump(failed_urls, open("go_ssi_failed_downloads.json", "w"), indent=2)

In [ ]:
edr = EDR("OPN0115")
edr = EDR("G8G0021")
edr = EDR("OPN0138")

In [ ]:
edr.download(refresh=True)

In [ ]:
edr.failed_urls

In [ ]:
edr.meta

In [ ]:
ind[ind["FILE_SPECIFICATION_NAME"].str.contains("C0394533078R", na=False)]


In [ ]:
edr.download_url

In [ ]:
ind.query("IMAGE_ID == 'OPN0115'").T